## Preliminaries

<br>

### Packages

In [ ]:
import os
import signal

<br>
<br>

#### Install libspatialindex

* Future: `export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:${directory}lib/`

In [ ]:
%%bash

apt-get install -qq curl g++ make

# Parameters
version=1.9.2
package=spatialindex-src-$version
directory="/usr/local/"

# Download & unpack tar
curl -L https://github.com/libspatialindex/libspatialindex/releases/download/$version/$package.tar.gz | tar xz

# Switch directory
cd $package

# make
cmake -DCMAKE_INSTALL_PREFIX=$directory .
make
make install

cp ${directory}lib/libspatialindex* /lib/

<br>
<br>

#### Install RTree

In [ ]:
%%bash

package=Rtree-0.9.4

curl -L https://files.pythonhosted.org/packages/56/6f/f1e91001d5ad9fa9bed65875152f5a1c7955c5763168cae309546e6e9fda/$package.tar.gz | tar xz

cd $package && /usr/local/bin/python setup.py install

In [ ]:
!rm -rf Rtree-0.9.4 && rm -rf spatialindex-src-1.9.2

<br>
<br>

#### Restart

* `os.kill(os.getpid(), signal.SIGKILL.value)`
* `os.system('systemctl reboot -i')`
* !ps

In [ ]:
!kill -SIGKILL `pgrep -f jupyter-noteboo`

<br>
<br>

#### GeoPandas

In [ ]:
!pip install geopandas

     |████████████████████████████████| 931kB 2.7MB/s 
     |████████████████████████████████| 10.9MB 14.4MB/s 
     |████████████████████████████████| 14.7MB 257kB/s 


<br>
<br>

### Libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import rtree
import numpy as np

<br>
<br>

### Classes & Functions

In [ ]:
class Geocoders:

    @staticmethod
    def simple(i):
        try:
            return gpd.tools.geocode(i, provider='nominatim', user_agent='spatial.analysis').loc[0, :]
        except:
            return pd.Series({'geometry': None, 'address': None})


In [ ]:
class GeoPandasCoders:

    def __init__(self):

        self.provider = 'nominatim'
        self.user_agent = 'spatial.analysis'

    def via(self, data: str):
        """
        :param data: An address string whose latitude & longitude coordinates will be searched-for
        :return:
            A pandas.core.series.Series of geopy.Location objects .. if the address is
            located, otherwise None.
        """

        try:
            return gpd.tools.geocode(data, provider=self.provider, user_agent=self.user_agent).loc[0, :]
        except:
            return pd.Series({'geometry': None, 'address': None})

    def geocoding(self, data: gpd.GeoDataFrame, field: str):
        """
        :param data: A series of addresses whose latitude & longitude coordinates will be searched-for
        :return:
            A GeoDataFrame of ... consisting of the address' geometry & address details; None
            if not found ...
        """

        instances = data.copy()

        estimates = instances.apply(lambda x: self.via(x[field]), axis=1 )
        estimates.dropna(axis=0, how='any', inplace=True)

        instances = instances.join(estimates, how='inner')

        instances['latitude'] = instances.geometry.y
        instances['longitude'] = instances.geometry.x

        return instances


<br>
<br>

## Explore

<br>

### Single Place

In [ ]:
place = 'sabine mfg.'
setup = gpd.GeoDataFrame(data={'place': [place]})

<br>
<br>

The coördinates of

* `setup.join( setup.apply(lambda x: Geocoders().simple(x['place']), axis=1) )`


In [ ]:
sample = setup.apply(lambda x: Geocoders().simple(x['place']), axis=1)
sample

,geometry,address
0,None,None


In [ ]:
sample = setup.join(sample)
sample

,place,geometry,address
0,sabine mfg.,None,None


In [ ]:
type(sample)

geopandas.geodataframe.GeoDataFrame

<br>

Points

In [ ]:
points = sample.geometry
latitude = points.y.values[0]
longitude = points.x.values[0]

In [ ]:
longitude

nan

<br>

Altogether

In [ ]:
GeoPandasCoders().geocoding(data=setup, field='place')

,place,geometry,address,latitude,longitude


<br>
<br>

### Places

<br>

#### Request latitudes & longitudes

In [ ]:
field = 'place'

universities = gpd.GeoDataFrame(
    data={field: ['The University of Edinburgh', 'The University of Cambridge', 'MIT', 'stjhn']}
)

estimates = universities.apply(lambda x: Geocoders().simple(x[field]), axis=1)
estimates.dropna(axis=0, how='any', inplace=True)

estimates

,geometry,address
0,POINT (-3.18841 55.94413),"University of Edinburgh, Lothian Street, Pleas..."
1,POINT (0.11974 52.19985),"Fitzwilliam Museum, Trumpington Street, Newnha..."
2,POINT (-71.09568 42.35840),"Massachusetts Institute of Technology, Portlan..."


<br>

#### Join

In [ ]:
data = universities.join(estimates, how='inner')
type(data)

geopandas.geodataframe.GeoDataFrame

<br>
<br>

#### DataFrame / GeoDataFrame

If the variable **data** is a DataFrame, rather than a GeoDataFrame, then the expressions


```python
    data['latitude'] = data.geometry.apply(lambda x: x.y)
    data['longitude'] = data.geometry.apply(lambda x: x.x )
    data
```

can be used to explicitly set **latutude** & **logitude** fields.  The expression

```python
    institutions = gpd.GeoDataFrame(data=data, geometry=data.geometry)
```

creates a GeoDataFrame from the DataFrame, and

```python
    institutions.crs = 'epsg:4326'
```

sets a coördinate reference system.

<br>
<br>


In [ ]:
data.crs == None

True

In [ ]:
data['latitude'] = data.geometry.y
data['longitude'] = data.geometry.x
data.crs = 'epsg:4326'
data

,place,geometry,address,latitude,longitude
0,The University of Edinburgh,POINT (-3.18841 55.94413),"University of Edinburgh, Lothian Street, Pleas...",55.944128,-3.188406
1,The University of Cambridge,POINT (0.11974 52.19985),"Fitzwilliam Museum, Trumpington Street, Newnha...",52.199852,0.119739
2,MIT,POINT (-71.09568 42.35840),"Massachusetts Institute of Technology, Portlan...",42.358396,-71.095678


In [ ]:
type(data)

geopandas.geodataframe.GeoDataFrame

<br>

#### Altogether

In [ ]:
GeoPandasCoders().geocoding(data=universities, field=field)

,place,geometry,address,latitude,longitude
0,The University of Edinburgh,POINT (-3.18841 55.94413),"University of Edinburgh, Lothian Street, Pleas...",55.944128,-3.188406
1,The University of Cambridge,POINT (0.11974 52.19985),"Fitzwilliam Museum, Trumpington Street, Newnha...",52.199852,0.119739
2,MIT,POINT (-71.09568 42.35840),"Massachusetts Institute of Technology, Portlan...",42.358396,-71.095678


<br>

#### Map

In [ ]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for _, record in data.iterrows():
    folium.Marker([record['latitude'], record['longitude']], popup=record['place']).add_to(m)

# Display the map
m

<br>
<br>

### Merges & Spatial Joins

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)
europe.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,142257519,Europe,Russia,RUS,3745000.0,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516..."
1,5320045,Europe,Norway,-99,364700.0,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ..."
2,67106161,Europe,France,-99,2699000.0,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ..."
3,9960487,Europe,Sweden,SWE,498100.0,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300..."
4,9549747,Europe,Belarus,BLR,165400.0,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372..."


In [ ]:
type(europe)

geopandas.geodataframe.GeoDataFrame

<br>

Slicing

In [ ]:
europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [ ]:
type(europe_stats)

pandas.core.frame.DataFrame

In [ ]:
type(europe_boundaries)

geopandas.geodataframe.GeoDataFrame

<br>

Merge

In [ ]:
europe_boundaries.merge(europe_stats, on='name').head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516...",142257519,3745000.0
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ...",5320045,364700.0
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ...",67106161,2699000.0
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300...",9960487,498100.0
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372...",9549747,165400.0


<br>

Spatial Join

In [ ]:
if not data.crs == europe.crs:
    data.crs = '{}'.format(europe.crs)

In [ ]:
join = gpd.sjoin(data, europe, how='inner', op='within')
join

,place,geometry,address,latitude,longitude,index_right,pop_est,continent,name,iso_a3,gdp_md_est
0,The University of Edinburgh,POINT (-3.18841 55.94413),"University of Edinburgh, Lothian Street, Pleas...",55.944128,-3.188406,28,64769452,Europe,United Kingdom,GBR,2788000.0
1,The University of Cambridge,POINT (0.11974 52.19985),"Fitzwilliam Museum, Trumpington Street, Newnha...",52.199852,0.119739,28,64769452,Europe,United Kingdom,GBR,2788000.0
